In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score

## Functions

In [2]:
def drop_vars(df):
    columns_to_drop = [
        # Those we want to remove for all analyses
        'Date','Vodka Sold PC','Gin Sold PC', 'Rum Sold PC', 'Whiskey Sold PC', 'Tequila Sold PC', 'Other Alc Sold PC',
        'Gin Sold PROP','Rum Sold PROP','Whiskey Sold PROP','Tequila Sold PROP','Other Alc Sold PROP','Vodka Sold PROP',
        # Remove linearly dependent columns
        'is_weekend', 'Young Prop', 'LowIncome Prop', 'Other Prop',
        # remove target columns
        'Volume Sold PC', 'Bottles Sold PC', 'Sales Volume PC']
    df = df.drop(columns_to_drop, axis = 1)
    return df


# specify what columns to make dummies for 
def make_dummies(df, columns):
    new_cols = []
    for col in columns:
        if col == 'Year':
            # get dummies for year 
            year_dummies = pd.get_dummies(df['Year'], drop_first = True)
            year_dummies.columns = ["year_" + str(column) for column in year_dummies.columns]
            new_cols.append(year_dummies)
            df = df.drop(['Year'], axis = 1)
        elif col == 'Month':
            # get dummies for month
            month_dummies = pd.get_dummies(df['Month'], drop_first = True)
            month_dummies.columns = ["month_" + str(column) for column in month_dummies.columns]
            new_cols.append(month_dummies)
            df = df.drop(['Month'], axis = 1)
        elif col == 'County':
            # get dummies for county
            county_dummies = pd.get_dummies(df['County'], drop_first = True)
            new_cols.append(county_dummies)
            df = df.drop(['County'], axis = 1)
        elif col == 'DOW':
            dow_dummies = pd.get_dummies(df['DOW'], drop_first = True)
            new_cols.append(dow_dummies)
            df = df.drop(['DOW'], axis = 1)
            
    full_set = [df]+new_cols
    df = pd.concat(full_set, axis = 1)
    return df


def get_intermediate_matrix(df, response_df, numeric_columns, standardize=True):
    
    
    # if there are numeric columns:
    if numeric_columns is not None and standardize is True:
        non_numeric_columns = [column for column in df.columns if column not in numeric_columns]
        # standard scale non categorical columns
        ct = make_column_transformer(
                (StandardScaler(), numeric_columns)
                , remainder='passthrough')
        ct_array = ct.fit_transform(df)
        column_order = ['Intercept'] + numeric_columns + non_numeric_columns
        x_mat = np.asmatrix(ct_array) 
    # only categorical columns
    else:
        non_numeric_columns = [column for column in df.columns]
        column_order = ['Intercept'] + non_numeric_columns
        x_mat = np.asmatrix(df)   
        
    intercept_array = np.ones((x_mat.shape[0], 1))
    x_mat = np.concatenate((intercept_array, x_mat),1)
    response_mat = np.asmatrix(response_df).getT()
    
    matrix_without_target = x_mat
    matrix_with_target = np.concatenate((response_mat, x_mat), 1)
    return matrix_with_target, matrix_without_target, response_mat, column_order


def get_betas(x_mat, response_mat, reg_type, lambda_val=100):
    if reg_type == 'OLS':
        # ols beta estimates
        betas = np.matmul(np.matmul(np.matmul(x_mat.getT(), x_mat).getI(), x_mat.getT()), response_mat)
    elif reg_type == 'Ridge':
        lambda_mat = np.diag(np.full(x_mat.shape[1], lambda_val))
        betas = np.matmul(np.matmul((np.matmul(x_mat.getT(), x_mat)+lambda_mat).getI(), x_mat.getT()), response_mat)
    return betas


def show_beta_summary(betas, column_order):
    return pd.DataFrame({'Variable' : column_order, 'Coefficient' : betas.A1}).sort_values(by = 'Coefficient', ascending = False)

In [3]:
def run_cross_val(data, k, penalty="OLS", lam=100):
    # shuffle matrix
    np.random.shuffle(data)
    k_splits = np.array_split(data, k)
    r2_list = []
    rmse_list = []
    metrics = {}
    for i in range(k):
        # Get train/validation split
        validate = k_splits[i]
        train = np.concatenate([k_splits[j] for j in range(k) if j != i])
        
        y_train = train[:, 0]
        x_train = train[:, 1:]
        
        y_val = validate[:,0]
        x_val = validate[:, 1:]
        # Train the model on the training data
        if penalty == "OLS":
            # ols beta estimates
            betas = np.matmul(np.matmul(np.matmul(x_train.getT(), x_train).getI(), x_train.getT()), y_train)
        elif penalty == "Ridge":
            # make diagonal matrix for lambda
            lambda_mat = np.diag(np.full(x_train.shape[1], lam))
            # ols beta estimates
            betas = np.matmul(np.matmul((np.matmul(x_train.getT(), x_train)+lambda_mat).getI(), x_train.getT()), y_train)
        else:
            raise ValueError("not a recognizable penalty")
        
        # Make predictions on validation data
        y_pred = np.matmul(x_val, betas)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        rmse_list.append(rmse)
        r2 = r2_score(y_val, y_pred)
        r2_list.append(r2)
        
    metrics['avg r_2'] = np.mean(r2_list)
    metrics['avg rmse'] = np.mean(rmse_list)
    
    return metrics

In [4]:
# prepare the df to plot RMSE/lambda
def roc_df(matrix, lambda_list, k):
    df = {'Lambda': [], 'Avg. RMSE': []}
    for l in lambda_list:
        # print('Working on lambda {}'.format(l))
        metrics = run_cross_val(matrix, k, "Ridge")
        df['Lambda'].append(l)
        df['Avg. RMSE'].append(metrics['avg rmse'])
    return pd.DataFrame(df)

## Reading in Dataset

In [5]:
# reading in raw data
raw_df = pd.read_csv("harlech_data_proportions.csv", low_memory = False)

## Train Dev Test Split

In [6]:
# pulling out response variables
response_df = raw_df[['Volume Sold PC', 'Bottles Sold PC', 'Sales Volume PC']]
# dropping the variables we are currently not using
df = drop_vars(raw_df)
# 0.1 test split 
X_train, X_test, y_train, y_test = train_test_split(df, response_df, test_size=0.1, random_state=420)

# *** DO NOT TOUCH X_TEST AND Y_TEST UNTIL THE VERY END *** 
- I set a random state already, so results should be consistent

In [7]:
# X_train X_test, y_train, y_test are all regular dfs
X_train.head()

,County,Year,Poverty,Population,DOW,Month,White Prop,Black Prop,Native American Prop,Asian Prop,Pacific Prop,Two+ Prop,HighIncome Prop,MidIncome Prop,Middle-Old Prop,Middle-Young Prop,Old Prop
18073,O'BRIEN,2014,7.3,14045.0,Tuesday,2,0.947985,0.005109,0.002058,0.008445,0.000142,0.012347,0.128670,0.659689,0.295180,0.218584,0.364049
50415,IOWA,2018,8.4,16150.0,Monday,12,0.959950,0.005509,0.000248,0.003776,0.000000,0.027050,0.205333,0.630667,0.283241,0.231381,0.352249
59342,BLACK HAWK,2019,15.5,131089.0,Tuesday,12,0.830555,0.096151,0.003968,0.026993,0.002784,0.028836,0.209806,0.543299,0.226826,0.241214,0.319122
30060,BOONE,2015,10.1,26468.0,Tuesday,9,0.971857,0.008901,0.002045,0.003220,0.000379,0.012462,0.145129,0.643683,0.293791,0.245383,0.324965
17082,LINN,2013,9.8,216567.0,Monday,12,0.897466,0.042222,0.002433,0.019862,0.000152,0.028476,0.211278,0.585274,0.280189,0.277637,0.275762


In [8]:
y_train.head()

,Volume Sold PC,Bottles Sold PC,Sales Volume PC
18073,0.178702,0.146315,1.822865
50415,0.061034,0.061300,0.760484
59342,0.156045,0.240005,2.845622
30060,0.009056,0.008425,0.139549
17082,0.016383,0.018766,0.260010


## Training the Data using our Best Subsets of Features

### Bottles Sold Predictive

In [53]:
bottles_sold_df = X_train[['County', 'Year', 'Poverty ', 'DOW', 'Month', 'Pacific Prop']]
bottles_sold_bs = make_dummies(bottles_sold_df, ['County','DOW', 'Month', 'Year'])

bs_mat_with, bs_mat_without, bs_response_mat, bs_col_order = get_intermediate_matrix(bottles_sold_bs, 
                                                                                     y_train['Bottles Sold PC'], 
                                                                                     ['Poverty ', 'Pacific Prop'])

In [105]:
bs_columns = bottles_sold_bs.columns.tolist()
sunday_index = bs_columns.index('Sunday')

In [106]:
bottles_sold_betas = get_betas(bs_mat_without, bs_response_mat, "OLS")

In [108]:
# MAKING PREDICTIONS ON TEST DATA
bottles_sold_test_df = X_test[['County', 'Year', 'Poverty ', 'DOW', 'Month', 'Pacific Prop']]
bottles_sold_test_df = make_dummies(bottles_sold_test_df,  ['County','DOW', 'Month', 'Year'])

# fix issue where sunday doesn't exist for test data
bottles_sold_test_df.insert(sunday_index, 'Sunday', [0 for x in range(len(bottles_sold_test_df.index))])
# ensuring columns in same order
bottles_sold_test_df = bottles_sold_test_df[bottles_sold_bs.columns]

bs_test_mat_with, bs_test_mat_without, bs_test_response_mat, bs_test_col_order = get_intermediate_matrix(bottles_sold_test_df, 
                                                                                     y_test['Bottles Sold PC'], 
                                                                                     ['Poverty ', 'Pacific Prop'])


In [109]:
bottles_sold_pred = np.matmul(bs_test_mat_without, bottles_sold_betas)
bs_rmse = np.sqrt(mean_squared_error(bs_test_response_mat, bottles_sold_pred))

In [110]:
bs_rmse

0.04676887352917497

In [111]:
bs_r2 = r2_score(bs_test_response_mat, bottles_sold_pred)
bs_r2

0.3441732538104013

### Volume Sold Predictive

In [112]:
vol_sold_df = X_train[['County', 'Year', 'Poverty ', 'DOW', 'Month', 'Pacific Prop']]
vol_sold_bs = make_dummies(vol_sold_df, ['County','DOW', 'Month', 'Year'])

vs_mat_with, vs_mat_without, vs_response_mat, vs_col_order = get_intermediate_matrix(vol_sold_bs, 
                                                                                     y_train['Volume Sold PC'], 
                                                                                     ['Poverty ', 'Pacific Prop'])

vs_columns = vol_sold_bs.columns.tolist()
sunday_index = vs_columns.index('Sunday')

vol_sold_betas = get_betas(vs_mat_without, vs_response_mat, "OLS")

In [120]:
# MAKING PREDICTIONS ON TEST DATA
vol_sold_test_df = X_test[['County', 'Year', 'Poverty ', 'DOW', 'Month', 'Pacific Prop']]
vol_sold_test_df = make_dummies(vol_sold_test_df,  ['County','DOW', 'Month', 'Year'])

# fix issue where sunday doesn't exist for test data
vol_sold_test_df.insert(sunday_index, 'Sunday', [0 for x in range(len(vol_sold_test_df.index))])
# ensuring columns in same order
vol_sold_test_df = vol_sold_test_df[vs_columns]

vs_test_mat_with, vs_test_mat_without, vs_test_true, vs_test_col_order = get_intermediate_matrix(vol_sold_test_df, 
                                                                                     y_test['Volume Sold PC'], 
                                                                                     ['Poverty ', 'Pacific Prop'])

In [121]:
vol_sold_pred = np.matmul(vs_test_mat_without, vol_sold_betas)
vs_rmse = np.sqrt(mean_squared_error(vs_test_true, vol_sold_pred))
vs_r2 = r2_score(vs_test_true, vol_sold_pred)

In [122]:
{'Vol Sold RMSE': vs_rmse, 'Vol Sold R2': vs_r2}

{'Vol Sold RMSE': 0.046299636315973114, 'Vol Sold R2': 0.3540135133878174}

### Sales Volume Pred

In [119]:
sales_df = X_train[['County', 'Year', 'Poverty ', 'DOW', 'Month', 'White Prop']]
sales_bs = make_dummies(sales_df, ['County','DOW', 'Month', 'Year'])

sales_mat_with, sales_mat_without, sales_response_mat, sales_col_order = get_intermediate_matrix(sales_bs, 
                                                                                     y_train['Sales Volume PC'], 
                                                                                     ['Poverty ', 'White Prop'])

sales_columns = sales_bs.columns.tolist()
sunday_index = sales_columns.index('Sunday')

sales_betas = get_betas(sales_mat_without, sales_response_mat, "OLS")

In [123]:
# MAKING PREDICTIONS ON TEST DATA
sales_test_df = X_test[['County', 'Year', 'Poverty ', 'DOW', 'Month', 'White Prop']]
sales_test_df = make_dummies(sales_test_df,  ['County','DOW', 'Month', 'Year'])

# fix issue where sunday doesn't exist for test data
sales_test_df.insert(sunday_index, 'Sunday', [0 for x in range(len(sales_test_df.index))])
# ensuring columns in same order
sales_test_df = sales_test_df[sales_columns]

sales_test_mat_with, sales_test_mat_without, sales_test_true, sales_test_col_order = get_intermediate_matrix(sales_test_df, 
                                                                                     y_test['Sales Volume PC'], 
                                                                                     ['Poverty ', 'White Prop'])

In [124]:
sales_pred = np.matmul(sales_test_mat_without, sales_betas)
sales_rmse = np.sqrt(mean_squared_error(sales_test_true, sales_pred))
sales_r2 = r2_score(sales_test_true, sales_pred)

In [125]:
{'Sales Volume RMSE': sales_rmse, 'Sales Volume R2': sales_r2}

{'Sales Volume RMSE': 0.6555754614527641,
 'Sales Volume R2': 0.34898985963782336}

## Interpretive Models

### Bottles Sold Interpretive

In [126]:
bs_int_df = X_train[['County', 'Year', 'DOW', 'Month']]
bottles_sold_int = make_dummies(bs_int_df, ['County','DOW', 'Month', 'Year'])

bs_int_mat_with, bs_int_mat_without, bs_int_response_mat, bs_int_col_order = get_intermediate_matrix(bottles_sold_int, 
                                                                                     y_train['Bottles Sold PC'], 
                                                                                     None)

bs_columns = bottles_sold_int.columns.tolist()
sunday_index = bs_columns.index('Sunday')

bs_int_betas = get_betas(bs_int_mat_without, bs_int_response_mat, "OLS")

In [127]:
# MAKING PREDICTIONS ON TEST DATA
bs_int_test_df = X_test[['County', 'Year', 'DOW', 'Month']]
bs_int_test_df = make_dummies(bs_int_test_df,  ['County','DOW', 'Month', 'Year'])

# fix issue where sunday doesn't exist for test data
bs_int_test_df.insert(sunday_index, 'Sunday', [0 for x in range(len(bs_int_test_df.index))])
# ensuring columns in same order
bs_int_test_df = bs_int_test_df[bs_columns]

bs_int_test_mat_with, bs_int_test_mat_without, bs_int_test_true, bs_int_test_col_order = get_intermediate_matrix(bs_int_test_df, 
                                                                                     y_test['Bottles Sold PC'], 
                                                                                     None)

In [128]:
bs_int_pred = np.matmul(bs_int_test_mat_without, bs_int_betas)
bs_int_rmse = np.sqrt(mean_squared_error(bs_int_test_true, bs_int_pred))
bs_int_r2 = r2_score(bs_int_test_true, bs_int_pred)

{'Bottles Sold Interpretative RMSE': bs_int_rmse, 'Bottles Sold Interpretative R2': bs_int_r2}

{'Bottles Sold Interpretative RMSE': 0.04677518005421671,
 'Bottles Sold Interpretative R2': 0.34399637263523697}

In [131]:
bs_int_beta_summary = show_beta_summary(bs_int_betas, bs_int_col_order)
bs_int_beta_summary

,Variable,Coefficient
29,DICKINSON,0.206974
16,CERRO GORDO,0.068658
20,CLAY,0.048237
72,PAGE,0.047407
14,CASS,0.046040
...,...,...
37,GRUNDY,-0.033034
25,DAVIS,-0.036235
11,BUTLER,-0.038729
120,year_2017,-0.052217


In [132]:
bs_int_beta_summary.to_csv('bottles_sold_interpretive_summary.csv')

### Volume Sold Interpretative

In [133]:
vs_int_df = X_train[['County', 'Year', 'DOW', 'Month']]
vs_int_df = make_dummies(vs_int_df, ['County','DOW', 'Month', 'Year'])

vs_int_mat_with, vs_int_mat_without, vs_int_response_mat, vs_int_col_order = get_intermediate_matrix(vs_int_df, 
                                                                                     y_train['Volume Sold PC'], 
                                                                                     None)

vs_columns = vs_int_df.columns.tolist()
sunday_index = vs_columns.index('Sunday')

vs_int_betas = get_betas(vs_int_mat_without, vs_int_response_mat, "OLS")

In [134]:
# MAKING PREDICTIONS ON TEST DATA
vs_int_test_df = X_test[['County', 'Year', 'DOW', 'Month']]
vs_int_test_df = make_dummies(vs_int_test_df,  ['County','DOW', 'Month', 'Year'])

# fix issue where sunday doesn't exist for test data
vs_int_test_df.insert(sunday_index, 'Sunday', [0 for x in range(len(vs_int_test_df.index))])
# ensuring columns in same order
vs_int_test_df = vs_int_test_df[vs_columns]

vs_int_test_mat_with, vs_int_test_mat_without, vs_int_test_true, vs_int_test_col_order = get_intermediate_matrix(vs_int_test_df, 
                                                                                     y_test['Volume Sold PC'], 
                                                                                     None)

In [135]:
vs_int_pred = np.matmul(vs_int_test_mat_without, vs_int_betas)
vs_int_rmse = np.sqrt(mean_squared_error(vs_int_test_true, vs_int_pred))
vs_int_r2 = r2_score(vs_int_test_true, vs_int_pred)

{'Volume Sold Interpretative RMSE': vs_int_rmse, 'Volume Sold Interpretative R2': vs_int_r2}

{'Volume Sold Interpretative RMSE': 0.04630395918661186,
 'Volume Sold Interpretative R2': 0.35389287976721007}

In [136]:
vs_int_beta_summary = show_beta_summary(vs_int_betas,vs_int_col_order)
vs_int_beta_summary

,Variable,Coefficient
29,DICKINSON,0.219601
16,CERRO GORDO,0.067680
20,CLAY,0.057215
13,CARROLL,0.056491
73,PALO ALTO,0.055022
...,...,...
25,DAVIS,-0.032018
35,FREMONT,-0.032820
11,BUTLER,-0.034240
101,Sunday,-0.049126


In [137]:
vs_int_beta_summary.to_csv('volume_sold_interpretive_summary.csv')

### Sales Volume Interpretative

In [138]:
sales_int_df = X_train[['County', 'Year', 'DOW', 'Month']]
sales_int_df = make_dummies(sales_int_df, ['County','DOW', 'Month', 'Year'])

sales_int_mat_with, sales_int_mat_without, sales_int_response_mat, sales_int_col_order = get_intermediate_matrix(sales_int_df, 
                                                                                     y_train['Sales Volume PC'], 
                                                                                     None)

sales_columns = sales_int_df.columns.tolist()
sunday_index = sales_columns.index('Sunday')

sales_int_betas = get_betas(sales_int_mat_without, sales_int_response_mat, "OLS")

In [139]:
# MAKING PREDICTIONS ON TEST DATA
sales_int_test_df = X_test[['County', 'Year', 'DOW', 'Month']]
sales_int_test_df = make_dummies(sales_int_test_df,  ['County','DOW', 'Month', 'Year'])

# fix issue where sunday doesn't exist for test data
sales_int_test_df.insert(sunday_index, 'Sunday', [0 for x in range(len(sales_int_test_df.index))])
# ensuring columns in same order
sales_int_test_df = sales_int_test_df[sales_columns]

sales_int_test_mat_with, sales_int_test_mat_without, sales_int_test_true, sales_int_test_col_order = get_intermediate_matrix(sales_int_test_df, 
                                                                                     y_test['Sales Volume PC'], 
                                                                                     None)

In [140]:
sales_int_pred = np.matmul(sales_int_test_mat_without, sales_int_betas)
sales_int_rmse = np.sqrt(mean_squared_error(sales_int_test_true, sales_int_pred))
sales_int_r2 = r2_score(sales_int_test_true, sales_int_pred)

{'Sales Volume Interpretative RMSE': sales_int_rmse, 'Sales Volume Interpretative R2': sales_int_r2}

{'Sales Volume Interpretative RMSE': 0.6556291144778399,
 'Sales Volume Interpretative R2': 0.3488832964891405}

In [141]:
sales_int_beta_summary = show_beta_summary(sales_int_betas,sales_int_col_order)
sales_int_beta_summary

,Variable,Coefficient
29,DICKINSON,3.088562
16,CERRO GORDO,0.932936
13,CARROLL,0.806045
14,CASS,0.734572
20,CLAY,0.699591
...,...,...
35,FREMONT,-0.421860
25,DAVIS,-0.453781
11,BUTLER,-0.483682
120,year_2017,-0.669892


In [142]:
sales_int_beta_summary.to_csv('sales_volume_interpretive_summary.csv')

## Extra Descriptive Stuff

In [143]:
y_test.mean()

Volume Sold PC     0.054903
Bottles Sold PC    0.057403
Sales Volume PC    0.755673
dtype: float64

In [145]:
y_test.std()

Volume Sold PC     0.057610
Bottles Sold PC    0.057756
Sales Volume PC    0.812569
dtype: float64